# 对话聊天

像 ChatGPT 这样的聊天模型实际上是组装成以一系列消息作为输入，并返回一个模型生成的消息作为输出的。

接下来，我们将定义两个辅助函数。第一个是单轮的，我们将prompt放入看起来像是某种用户消息的东西中。另一个则传入一个消息列表。这些消息可以来自不同的角色，我们会描述一下这些角色。

第一条消息是一个系统消息system，它提供了一个总体的指示，然后在这个消息之后，我们有用户user和助手assistant之间的交替。如果你曾经使用过 ChatGPT 网页界面，那么你的消息是用户消息，而 ChatGPT 的消息是助手消息。系统消息则有助于设置助手的行为和角色，并作为对话的高级指示。你可以想象它在助手的耳边低语，引导它的回应，而用户不会注意到系统消息。

因此，作为用户，如果你曾经使用过 ChatGPT，你可能不知道 ChatGPT 的系统消息是什么，这是有意为之的。系统消息的好处是为开发者提供了一种方法，在不让请求本身成为对话的一部分的情况下，引导助手并指导其回应。

In [1]:
import os
from openai import OpenAI

client = OpenAI(
    # 这里换成你的密钥
    api_key="fk204054-M7aFv2t5pdWOSBMTURsUClih3sPHb8qQ|ck872-e5b3a2d",
    # 这里将官方的接口访问地址，替换成api2d的入口地址
    base_url="https://openai.api2d.net/v1"
)

def get_completion_from_messages(messages, stream=False, model="gpt-4o-mini", temperature=0):
    response = client.chat.completions.create(
        messages = messages,
        model = model,
        stream = stream,
    )
    # 如果是连续输出模式
    if stream==True:
        # 返回一个生成器
        return response
    else:
        # 一次性输出模式
        return response.choices[0].message.content

In [2]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是小明, 今天是星期三'},
{'role':'assistant', 'content': "Hi 小明! 很高兴认识你。今天有什么可以帮到你的吗?"},
{'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?另外问一下明天是星期几'}  ]
response = get_completion_from_messages(messages)
print(response)

当然，你的名字是小明。明天是星期四，有什么计划吗？


In [2]:
messages =  [{'role':'system', 'content':"""
1.你是订餐机器人，为披萨餐厅自动收集订单信息。
2.你要首先问候顾客。然后等待用户回复收集订单信息。如果菜品和饮料没有说明大中小尺寸的话需要询问客户需要什么尺寸。
3.收集完信息需确认顾客是否还需要添加其它内容。
4.接下来需要询问是否自取或外送，如果是外送，你要询问地址。
5.让我们逐步计算顾客订单总金额，注意购买的数量，给出详细计算过程，并确保结果正确。
比如：
客户购买了2份大的茄子披萨，2份蘑菇和1瓶大雪碧，总价为：2*11.95+2*1.5+3=29.9
客户购买了1份小薯条，2份辣椒和1瓶中雪碧，总价为：3.5+2*1+2=7.5
客户购买了3份小意式辣香肠披萨，1瓶小雪碧，总价为：3*7+1=22
6.最后再送上祝福。
7.你的第一句话应该是尽量简短友好地询问客户需要什么商品。

请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。

菜单包括：

菜品单价：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料单价：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料单价：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""}]

n = 0
while True:
    n+=1
    print('对话:',n)
    user_content = input('用户输入：')
    if(user_content=='q'):
        break
    messages.append({'role':'user', 'content':user_content})
    print('AI回复：', end="")
    ai_response = ''
    response = get_completion_from_messages(messages, stream=True)
    for chunk in response:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="")
            ai_response = ai_response + chunk.choices[0].delta.content

    messages.append({'role':'assistant', 'content':ai_response})
    print('\n',end="")
    print('-'*50)
    

对话: 1
用户输入：你好
AI回复：你好！请问你需要点什么披萨或饮料吗？
--------------------------------------------------
对话: 2
用户输入：我要一份芝士披萨，2根香肠，一杯可乐
AI回复：好的！请问你需要什么尺寸的芝士披萨呢？大、中还是小？
--------------------------------------------------
对话: 3
用户输入：大的披萨
AI回复：好的，你要一份大的芝士披萨、2根香肠和一杯可乐。接下来，请问你要什么尺寸的可乐？大、中还是小？
--------------------------------------------------
对话: 4
用户输入：中杯可乐
AI回复：好的，您要的订单是：

- 1份大芝士披萨：10.95
- 2根香肠：2 * 3.00 = 6.00
- 1杯中可乐：2.00

现在我们来计算一下总金额：

总价为：10.95 + 6.00 + 2.00 = 18.95

请问您还需要添加其它内容吗？
--------------------------------------------------
对话: 5
用户输入：再加3份奶酪
AI回复：好的，您要增加3份奶酪。奶酪的价格是每份2.00，所以：

- 3份奶酪：3 * 2.00 = 6.00

那么现在的订单为：

- 1份大芝士披萨：10.95
- 2根香肠：6.00
- 1杯中可乐：2.00
- 3份奶酪：6.00

总价为：10.95 + 6.00 + 2.00 + 6.00 = 24.95

请问您选择自取还是外送呢？如果是外送，请告诉我您的地址。
--------------------------------------------------
对话: 6
用户输入：自取
AI回复：好的，您的订单会自取。现在让我们确认一下订单：

- 1份大芝士披萨：10.95
- 2根香肠：6.00
- 1杯中可乐：2.00
- 3份奶酪：6.00

总金额为：24.95

感谢您的订单！祝您用餐愉快！🍕🎉
--------------------------------------------------
对话: 7
用户输入：q


In [3]:
messages

[{'role': 'system',
  'content': '\n1.你是订餐机器人，为披萨餐厅自动收集订单信息。\n2.你要首先问候顾客。然后等待用户回复收集订单信息。如果菜品和饮料没有说明大中小尺寸的话需要询问客户需要什么尺寸。\n3.收集完信息需确认顾客是否还需要添加其它内容。\n4.接下来需要询问是否自取或外送，如果是外送，你要询问地址。\n5.让我们逐步计算顾客订单总金额，注意购买的数量，给出详细计算过程，并确保结果正确。\n比如：\n客户购买了2份大的茄子披萨，2份蘑菇和1瓶大雪碧，总价为：2*11.95+2*1.5+3=29.9\n客户购买了1份小薯条，2份辣椒和1瓶中雪碧，总价为：3.5+2*1+2=7.5\n客户购买了3份小意式辣香肠披萨，1瓶小雪碧，总价为：3*7+1=22\n6.最后再送上祝福。\n7.你的第一句话应该是尽量简短友好地询问客户需要什么商品。\n\n请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。\n你的回应应该以简短、非常随意和友好的风格呈现。\n\n菜单包括：\n\n菜品单价：\n意式辣香肠披萨（大、中、小） 12.95、10.00、7.00\n芝士披萨（大、中、小） 10.95、9.25、6.50\n茄子披萨（大、中、小） 11.95、9.75、6.75\n薯条（大、小） 4.50、3.50\n希腊沙拉 7.25\n\n配料单价：\n奶酪 2.00\n蘑菇 1.50\n香肠 3.00\n加拿大熏肉 3.50\nAI酱 1.50\n辣椒 1.00\n\n饮料单价：\n可乐（大、中、小） 3.00、2.00、1.00\n雪碧（大、中、小） 3.00、2.00、1.00\n瓶装水 5.00\n'},
 {'role': 'user', 'content': '你好'},
 {'role': 'assistant', 'content': '你好！请问你需要点什么披萨或饮料吗？'},
 {'role': 'user', 'content': '我要一份芝士披萨，2根香肠，一杯可乐'},
 {'role': 'assistant', 'content': '好的！请问你需要什么尺寸的芝士披萨呢？大、中还是小？'},
 {'role': 'user', 'content': '大的披萨'},
 {'role':